In [1]:
!git clone https://github.com/Sandipan99/POLAR.git

Cloning into 'POLAR'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 181 (delta 78), reused 115 (delta 35), pack-reused 0
Receiving objects: 100% (181/181), 11.08 MiB | 15.94 MiB/s, done.
Resolving deltas: 100% (78/78), done.


In [2]:
!wget https://nlp.stanford.edu/data/glove.42B.300d.zip
!unzip glove*.zip


--2022-03-20 22:25:22--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-03-20 22:25:22--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  5.02MB/s    in 5m 54s  

2022-03-20 22:31:16 (5.06 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [3]:
import gensim
from numpy import linalg
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath

!pip install ipython-autotime
%load_ext autotime

time: 184 µs (started: 2022-03-20 22:32:11 +00:00)


In [4]:
model_glove = glove2word2vec('glove.42B.300d.txt','gensim_glove_300d.txt') ## needed for glove embeddings only 


time: 33.1 s (started: 2022-03-20 22:32:35 +00:00)


In [5]:
model_glove = gensim.models.KeyedVectors.load_word2vec_format("gensim_glove_300d.txt", binary=False)

time: 5min 10s (started: 2022-03-20 22:33:08 +00:00)


In [6]:
def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    temp_file.close()

time: 8.5 ms (started: 2022-03-20 22:38:49 +00:00)


In [7]:
generate_norm_embedding(model_glove,'glove_norm_300.mod')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1917494 [00:00<?, ?it/s]

time: 51.9 s (started: 2022-03-20 22:38:50 +00:00)


In [8]:
model_gn = gensim.models.KeyedVectors.load_word2vec_format('glove_norm_300.mod',binary=True) # ss


time: 29.8 s (started: 2022-03-20 22:39:51 +00:00)


In [9]:
current_model = model_gn 


time: 952 µs (started: 2022-03-20 22:40:53 +00:00)


In [11]:
%cd POLAR

/content/POLAR
time: 10.4 ms (started: 2022-03-20 22:41:53 +00:00)


In [18]:

list_antonym = []

with open('Antonym_sets/LenciBenotto.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))


with open('Antonym_sets/LenciBenotto.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
with open('Antonym_sets/EVALution.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))

with open('Antonym_sets/EVALution.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
                
list_antonym = list(dict.fromkeys(list_antonym).keys())

time: 49 ms (started: 2022-03-20 22:44:52 +00:00)


In [64]:
import pickle

filename = 'ticker_embedding'
outfile = open(filename,'wb')

pickle.dump(ticker_word_embedding,outfile)
outfile.close()

time: 11.3 ms (started: 2022-03-20 23:21:51 +00:00)


In [21]:
similarity_matrix = defaultdict(list)
for each_pair in tqdm(list_antonym):
    word1 = each_pair[0]
    word2 = each_pair[1]
    if word1 < word2:
        similarity_matrix[word1].append(word2)
    else:
        similarity_matrix[word2].append(word1)
    
all_similarity = defaultdict(dict)
for each_key in tqdm(similarity_matrix):
    for each_value in similarity_matrix[each_key]:
#         cosine_similarity([current_model[each_key]]
        all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
    
final_antonym_list = []
for index_counter, each_key in enumerate(tqdm(all_similarity)):
#     print(each_key,all_similarity[each_key])
    listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
#     print(listofTuples)
    final_antonym_list.append((each_key, listofTuples[0][0]))
print(len(final_antonym_list))

list_antonym = final_antonym_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/3110 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/1468 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1468 [00:00<?, ?it/s]

1468
time: 3.64 s (started: 2022-03-20 22:45:39 +00:00)


In [25]:
num_antonym = 1468


time: 10 ms (started: 2022-03-20 22:47:11 +00:00)


In [26]:
## Find the antonym difference vectors
antonymy_vector = []
for each_word_pair in list_antonym:
    antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
antonymy_vector = np.array(antonymy_vector)
print(antonymy_vector.shape)

(1468, 300)
time: 26.4 ms (started: 2022-03-20 22:47:12 +00:00)


In [27]:
import random

from scipy.spatial.distance import cosine as scipy_cosine
random.seed(42)

t1 = np.array(antonymy_vector)
dimension_similarity_matrix = defaultdict(dict)
for index_1, each_dim1 in enumerate(tqdm(t1)):
    for index_2, each_dim2 in enumerate(t1):
        dimension_similarity_matrix[index_1][index_2] = abs(1-scipy_cosine(each_dim1, each_dim2))
        
        
def get_set_score(final_list, each_dim):
    final_output = 0.0
    for each_vec in final_list:
        final_output += dimension_similarity_matrix[each_vec][each_dim]
    return final_output/(len(final_list))
        
def select_subset_dimension(dim_vector, num_dim):
    working_list = np.array(dim_vector)
    
    working_position_index = [i for i in range(working_list.shape[0])]
    final_position_index = []
    

    print('working list is ready, shape', working_list.shape)
    sel_dim = random.randrange(0, working_list.shape[0])

    final_position_index.append(sel_dim)
    
    working_position_index.remove(sel_dim)

    for test_count in tqdm(range(num_dim-1)):
        min_dim = None
        min_score = 1000
        for temp_index, each_dim in enumerate(working_position_index):
#             print(each_dim)
            temp_score = get_set_score(final_position_index, each_dim)
            if temp_score< min_score:
                min_score= temp_score
                min_dim = each_dim
        print(test_count,min_dim)
        final_position_index.append(min_dim)
        working_position_index.remove(min_dim)
#         print(working_list.shape, len(final_list))
    return final_position_index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1468 [00:00<?, ?it/s]

time: 2min 59s (started: 2022-03-20 22:47:50 +00:00)


In [28]:
# use random dimension selection

random_antonym_vector = [i for i in range(len(antonymy_vector))]
random.shuffle(random_antonym_vector)
print(len(random_antonym_vector))

1468
time: 4.91 ms (started: 2022-03-20 22:51:10 +00:00)


In [33]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):
    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    print('New model size is',len(current_model.vocab), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model.vocab))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model.vocab))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model.vocab):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)

        
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()

time: 71.6 ms (started: 2022-03-20 22:52:20 +00:00)


In [34]:
def standard_normal_dist_model(model, new_filename):
    embedding_matrix = []
    embedding_vocab = []

    temp_file = open(new_filename,'wb')
    temp_file.write(str.encode(str(model.vectors.shape[0])+' '+str(model.vectors.shape[1])+'\n'))
    
    for each_word in tqdm(model.vocab):
        embedding_matrix.append(model[each_word])
        embedding_vocab.append(each_word)
    
    embedding_matrix = np.array(embedding_matrix)
    
    print('The shape of embedding matrix is {}'.format(embedding_matrix.shape))
    
    norm_embedding_matrix = (embedding_matrix - embedding_matrix.mean(0))/ embedding_matrix.std(0)
    
    for word_counter, each_word in enumerate(tqdm(embedding_vocab)):
#         assert each_word==embedding_vocab[word_counter],'Not matching!!!'
        
        temp_file.write(str.encode(each_word+' '))
        new_vector = norm_embedding_matrix[word_counter]
        temp_file.write(new_vector)
        temp_file.write(str.encode('\n'))
        
    del embedding_matrix
    del embedding_vocab
    temp_file.close()

time: 23.6 ms (started: 2022-03-20 22:52:33 +00:00)


In [80]:
def generate_embedding_path(current_model, embedding_path, binary, antonym_vector, curr_dim):
    curr_antonym_vector = antonymy_vector[antonym_vector[:curr_dim]]
    curr_antonymy_vector_inverse = np.linalg.pinv(np.transpose(curr_antonym_vector))
    new_embedding_dict = transform_to_antonym_space(current_model, embedding_path, binary,curr_antonymy_vector_inverse)

    return new_embedding_dict

time: 4.99 ms (started: 2022-03-20 23:27:33 +00:00)


In [36]:
import subprocess
dim_size = 500 # Number of POLAR dimenions
antonym_vector_method = random_antonym_vector # orthogonal_antonymy_vector, variance_antonymy_vector

time: 2.2 ms (started: 2022-03-20 22:53:23 +00:00)


In [68]:
generate_embedding_path(ticker_word_embedding, 'test_run',True ,random_antonym_vector,500)

New model size is 892 500


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/892 [00:00<?, ?it/s]

time: 274 ms (started: 2022-03-20 23:23:56 +00:00)


In [37]:
curr_dim = dim_size

embedding_path = 'Downstream Task/embeddings/rand_antonym_'+str(curr_dim)+'.bin'
generate_embedding_path(current_model, embedding_path,True,random_antonym_vector, curr_dim)

print('loading the model')
temp_model = gensim.models.KeyedVectors.load_word2vec_format(embedding_path, binary=True)
print('loading done..')

std_nrml_embedding_path = 'Downstream Task/embeddings/rand_antonym_gl_'+str(curr_dim)+'_StdNrml.bin'
standard_normal_dist_model(temp_model, std_nrml_embedding_path)

######### Task specific code....
command_list = ['python3', 
                    'Downstream Task/TREC/classify_task.py',
                     std_nrml_embedding_path,
                     '2',
                     'Downstream Task/TREC/data/qa_train_X.pickle',
                     'Downstream Task/TREC/data/qa_train_y.pickle',
                     'Downstream Task/TREC/data/qa_val_X.pickle',
                     'Downstream Task/TREC/data/qa_val_y.pickle',
                     'Downstream Task/TREC/data/qa_test_X.pickle',
                     'Downstream Task/TREC/data/qa_test_y.pickle'
                    ]
result = subprocess.run(command_list, stdout=subprocess.PIPE)
print(result.stdout)
print('finished sentiment task')

New model size is 1917494 500


FileNotFoundError: ignored

time: 539 ms (started: 2022-03-20 22:54:50 +00:00)


In [45]:
import pandas as pd

time: 1.24 ms (started: 2022-03-20 23:11:46 +00:00)


In [54]:
company = pd.read_csv('glove-ticker-fortune1000-us.csv')
ticker_list = company['Ticker'].str.lower()


time: 34.1 ms (started: 2022-03-20 23:14:43 +00:00)


In [62]:
ticker_word_embedding = dict()
for ticker in ticker_list:
  ticker_word_embedding[ticker] = current_model[ticker]

time: 6.63 ms (started: 2022-03-20 23:20:38 +00:00)


In [81]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):

    temp_dict = dict()

    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    print('New model size is',len(current_model), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)
        temp_dict[each_word] = new_vector
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()
    return temp_dict

time: 64.1 ms (started: 2022-03-20 23:27:41 +00:00)


In [82]:
ticker_new_embedding = generate_embedding_path(ticker_word_embedding, 'test_run',True ,random_antonym_vector,500)

New model size is 892 500


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/892 [00:00<?, ?it/s]

time: 293 ms (started: 2022-03-20 23:27:44 +00:00)


In [76]:
list_antonym[random_antonym_vector[0]]

('convict', 'pardon')

time: 4.5 ms (started: 2022-03-20 23:25:43 +00:00)


In [85]:
ticker_new_embedding.keys()

dict_keys(['wmt', 'amzn', 'xom', 'aapl', 'cvs', 'brka', 'unh', 'mck', 't', 'abc', 'googl', 'f', 'ci', 'cost', 'cvx', 'cah', 'jpm', 'gm', 'wba', 'vz', 'msft', 'mpc', 'kr', 'fnma', 'bac', 'hd', 'psx', 'cmcsa', 'antm', 'wfc', 'c', 'vlo', 'ge', 'dell', 'jnj', 'tgt', 'ibm', 'rtx', 'ba', 'fmcc', 'cnc', 'ups', 'low', 'intc', 'fb', 'fdx', 'met', 'dis', 'pg', 'pep', 'hum', 'pru', 'adm', 'aci', 'syy', 'lmt', 'hpq', 'et', 'gs', 'ms', 'cat', 'csco', 'pfe', 'hca', 'aig', 'axp', 'dal', 'mrk', 'aal', 'chtr', 'all', 'bby', 'ual', 'dow', 'tsn', 'tjx', 'orcl', 'gd', 'de', 'nke', 'pgr', 'ko', 'tecd', 'int', 'hon', 'cop', 'exc', 'noc', 'cof', 'pagp', 'abbv', 'epd', 'mmm', 'abt', 'trv', 'pm', 'hpe', 'arw', 'viac', 'dg', 'usb', 'sbux', 'bmy', 'mdlz', 'pcar', 'tmo', 'm', 'jbl', 'khc', 'duk', 'tsla', 'pbf', 'qcom', 'ngl', 'cbre', 'bkr', 'snx', 'dltr', 'cmi', 'unfi', 'mu', 'amgn', 'pag', 'v', 'avgo', 'nue', 'gild', 'luv', 'hal', 'lumn', 'ip', 'lly', 'afl', 'len', 'oxy', 'unp', 'rad', 'pnc', 'dd', 'so', 'an', '

time: 3.05 ms (started: 2022-03-20 23:28:08 +00:00)


In [96]:
ticker_new_embedding['amzn'][0]

0.02343967

time: 5.87 ms (started: 2022-03-20 23:31:40 +00:00)


In [93]:
antonym_500 = [list_antonym[x] for x in random_antonym_vector[:500]]

time: 1.86 ms (started: 2022-03-20 23:30:49 +00:00)


In [95]:
antonym_500[0]

('convict', 'pardon')

time: 8.7 ms (started: 2022-03-20 23:31:04 +00:00)


In [116]:
def make_polar_dict(company_name, antonym, embedding, top_n = False, n = 10):
  temp_dict = dict()
  temp_polar = embedding[company_name]

  if top_n:
    idx = np.argsort([abs(x) for x in temp_polar])[-n:]
    for i in idx:
      print(antonym[i],temp_polar[i],'\n')


  if len(antonym) == len(temp_polar):
    for a in range(len(antonym)):
      temp_dict[antonym[a]] = temp_polar[a]
    return temp_dict




time: 9.92 ms (started: 2022-03-20 23:51:24 +00:00)


In [117]:
jpm_polar = make_polar_dict('jpm', antonym_500, ticker_new_embedding, True)

('impossible', 'probable') -0.1047612 

('distribution', 'hoarding') -0.10681351 

('hail', 'snow') -0.10899254 

('loathe', 'pine') 0.10918276 

('kill', 'reanimate') 0.11325266 

('sidewalk', 'street') -0.116916135 

('school', 'work') -0.12668115 

('apple', 'tomato') 0.13062128 

('practice', 'theory') 0.13276422 

('we', 'you') 0.14179388 

time: 6.65 ms (started: 2022-03-20 23:51:28 +00:00)


In [124]:
amzn_polar = make_polar_dict('amzn', antonym_500, ticker_new_embedding, True)

('lop', 'secure') -0.100378804 

('mate', 'split') -0.10372001 

('cell', 'freedom') -0.109427296 

('handle', 'spout') -0.11193576 

('breathless', 'pulmonary') 0.1210895 

('dash', 'dot') -0.12288282 

('relax', 'toil') -0.12576644 

('impossible', 'probable') -0.1387733 

('loathe', 'pine') 0.15112714 

('apple', 'tomato') 0.20126186 

time: 2.97 ms (started: 2022-03-20 23:53:22 +00:00)


In [119]:
wmt_polar = make_polar_dict('wmt', antonym_500, ticker_new_embedding, True)

('impossible', 'probable') -0.10529984 

('lop', 'secure') -0.10963182 

('consolidation', 'distribution') -0.114220284 

('neglect', 'pontificate') 0.1146768 

('recycle', 'waste') 0.11501446 

('slowness', 'velocity') 0.11684404 

('sidewalk', 'street') -0.11836764 

('relax', 'toil') -0.122109905 

('loathe', 'pine') 0.16148268 

('apple', 'tomato') 0.1854346 

time: 16 ms (started: 2022-03-20 23:52:01 +00:00)


In [123]:
fb_polar = make_polar_dict('fb', antonym_500, ticker_new_embedding, True)

('appealing', 'gross') 0.11022391 

('bush', 'tree') -0.11114963 

('adult', 'small') -0.11564736 

('impossible', 'probable') -0.12567075 

('boss', 'worker') 0.12983803 

('bumpy', 'level') -0.13134277 

('careless', 'provident') -0.13689688 

('adventure', 'lazy') -0.14274842 

('cotton', 'wool') 0.14281982 

('orange', 'tomato') 0.14545166 

time: 13.5 ms (started: 2022-03-20 23:53:11 +00:00)


In [121]:
wfc_polar = make_polar_dict('wfc', antonym_500, ticker_new_embedding, True) # wells fargo

('earth', 'venus') 0.10587819 

('rapid', 'slow') 0.106058866 

('everybody', 'nobody') -0.10778228 

('sidewalk', 'street') -0.113352574 

('intense', 'mild') 0.11467203 

('orange', 'tomato') -0.11647428 

('detail', 'general') -0.12665822 

('cement', 'mortar') 0.12883052 

('practice', 'theory') 0.13569698 

('recycle', 'waste') 0.13713722 

time: 18.2 ms (started: 2022-03-20 23:52:44 +00:00)
